In [1]:
#!pip install pandas pillow
#asenna tarvittaessa

/opt/conda/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
import pandas as pd
from PIL import Image
from os import path

# Define the directory where images are stored
#image_directory = "../../data/kaivot/kuvat/kaivot/kaivot_kuvat"
image_directory = "../data/kuvat"
# Load the CSV file
#csv_file_path = "runs/detect/exp20/predictions.csv"  
csv_file_path ="../yolov5/runs/detect/exp5/predictions.csv" # Adjust the path as needed
data = pd.read_csv(csv_file_path)

# Remove duplicate rows if any
data = data.drop_duplicates()

# Add Latitude and Longitude columns if they don't exist
if 'Latitude' not in data.columns:
    data['Latitude'] = None
if 'Longitude' not in data.columns:
    data['Longitude'] = None

# Function to extract GPS coordinates from image EXIF data
def get_coordinates(image_path):
    try:
        with Image.open(image_path) as img:
            exif_data = img._getexif()
            if exif_data:
                gps_info = exif_data.get(34853)
                if gps_info:
                    latitude = gps_info.get(2)
                    longitude = gps_info.get(4)
                    return latitude, longitude
    except Exception as e:
        print(f"Error reading {image_path}: {e}")
    return None, None

# Process each row in the DataFrame
for index, row in data.iterrows():
    if row['Prediction'] in ['umpinainen', 'ritilä']:
        image_path = path.join(image_directory, row['Image Name'])
        if path.exists(image_path):
            latitude, longitude = get_coordinates(image_path)
            data.at[index, 'Latitude'] = latitude
            data.at[index, 'Longitude'] = longitude
        else:
            print(f"Image not found: {row['Image Name']}")

# Save the updated DataFrame back to CSV
data.to_csv("updated_predictions.csv", index=False)

print("Metadata extraction and CSV update complete.")


Metadata extraction and CSV update complete.
